# NNIA 18/19 Project 3:  Regularization and Data Augmentation

## Deadline: 26. January 2019, 23:59

## 1. Dropout$~$ (9 points)

In the following exercise you would use an already implemented **feed-forward network** using **only** Numpy. We will add here the dropout regualrization technique that you learned about in the lecture.

The goal is to implement dropout for layer 2 in the provided three-layers network.

Before you start, here is a simple introduction about dropout. Dropout is a regualization technique that performs random dropping of neurons with a certain probability at each mini-batch in training time. Introducing such randomness or noise at training is helpful to make the network not fully dependent on the training set and therefore, it could generalize better to other unseen data. 

A simple dropout implementation creates a mask ($r^{(l)}_j$) for every neuron $j$ of the hidden layer $l$ by drawing from a [Bernoulli distribution](https://en.wikipedia.org/wiki/Bernoulli_distribution) with probability $p$, where $p$ is the probability of keeping neurons. You can think of it as a coin flip for each neuron with a probability $p$ to determine if that neuron will be kept or not. For more information, refer to section 7.12 in Deep Learning book.

$$ r^{(l)}_j \sim Bernoulli(p) $$
This mask is then applied to the hidden layer output ($h^{(l)}$) to obtain the regularized hidden layer activation $\hat{h}^{(l)}$
$$ \hat{h}^{(l)} = r^{(l)} * h^{(l)}$$
However, such an implementation requires the layer to be multiplied by the dropout coefficient $p$ at evaluation time to balance the larger number of active units during testing.
$$ \hat{h}^{(l)} = p * h^{(l)}$$
Such an implementation requires the code to switch between different code blocks for forward-pass evaluation during training and testing. Hence, a smoother way to implement dropout is to use ***inverted dropout*** where the mask generated at the training is multiplied by the inverse of the dropout coefficient.
$$ r^{(l)}_j \sim Bernoulli(p) * \frac{1}{p}$$
This scheme allows the scaling to be learned during training and hence, no switching between code blocks is required. The idea of using inverted dropout is to provide a form of normalization; when dropping out neurons during training, we are now giving the network less neurons that what it will have in the test time. So, dividing by the keep_probs is done to scale the training so that we will not need to do anything in test time. 

a) The following code implements a simple one-hiddien-layer neural network similar to what you have done in project 2. Update this code to implement inverted dropout for a hidden layer size of 50 neurons (sections to be updated are marked with TODO). Make sure to handle forward and backward passes and handle inference time as well (5.5 points)


- Load **Fashion-MNIST** dataset and normalized it.

In [ ]:
import os
import struct
import numpy as np
import tensorflow as tf

fashion_mnist = tf.keras.datasets.fashion_mnist
(X_trainval, Y_trainval), (X_test, Y_test) = fashion_mnist.load_data()

In [ ]:
print(X_trainval.shape)

In [ ]:
X_trainval = np.reshape(X_trainval, (X_trainval.shape[0],  X_trainval.shape[1] *  X_trainval.shape[2]))
print('The X_trainval has the following shape:')
print('Rows: %d, columns: %d' % (X_trainval.shape[0], X_trainval.shape[1]))

In [ ]:
X_test = np.reshape(X_test, (X_test.shape[0],  X_test.shape[1] *  X_test.shape[2]))
print('The X_test has the following shape:')
print('Rows: %d, columns: %d' % (X_test.shape[0], X_test.shape[1]))

- Normalize the data. Subtract the mean and divide by the standard deviation.

In [ ]:
def data_normalization(X_trainval, X_test):
    feature_means = X_trainval.mean(axis = 0)
    feature_std = np.std(X_trainval, axis = 0)
    X_trainval_normalized = (X_trainval - feature_means) / feature_std
    X_test_normalized = (X_test - feature_means) / feature_std
    return X_trainval_normalized, X_test_normalized

In [ ]:
# The normalization should be done on X_train and X_test
# and normalized data should have the exactly same shape as the original data matrix.

X_trainval, X_test = data_normalization(X_trainval, X_test)
print(X_trainval.shape)
print(X_test.shape)

In [ ]:
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import expit
from __future__ import division


class MLP(object):
    """ Feedforward neural network / Multi-layer perceptron classifier.

    Parameters
    ------------
    n_output : int
        Number of output units, should be equal to the
        number of unique class labels.
        
    n_features : int
        Number of features (dimensions) in the target dataset.
        Should be equal to the number of columns in the X array.
        
    n_hidden : int (default: 30)
        Number of hidden units.
        
    l1 : float (default: 0.0)
        Lambda value for L1-regularization.
        No regularization if l1=0.0 (default)
        
    l2 : float (default: 0.0)
        Lambda value for L2-regularization.
        No regularization if l2=0.0 (default)
        
    epochs : int (default: 500)
        Number of passes over the training set.
        
    eta : float (default: 0.001)
        Learning rate.
        
    alpha : float (default: 0.0)
        Momentum constant.
        
    decrease_const : float (default: 0.0)
        Decrease constant. Shrinks the learning rate
        after each epoch via eta / (1 + epoch*decrease_const)
        
    shuffle : bool (default: True)
        Shuffles training data every epoch if True to prevent circles.
        
    minibatches : int (default: 1)
        Divides training data into k minibatches for efficiency.
        Normal gradient descent learning if k=1 (default).
        
    random_state : int (default: None)
        Set random state for shuffling and initializing the weights.
    
    dropout : float (default: 1.0, no dropout)
        Set the dropout coefficient (keep probability of neurons)

    Attributes
    -----------
    cost_ : list
      Sum of squared errors after each epoch.

    """

      
    def __init__(self, n_output, n_features, n_hidden=30,
                 l1=0.0, l2=0.0, epochs=50, eta=0.001,
                 alpha=0.0, decrease_const=0.0, shuffle=True,
                 minibatches=1, random_state=1, dropout = 1.0):

        self.n_output = n_output
        self.r = np.random.RandomState(random_state)
        self.n_features = n_features
        self.n_hidden = n_hidden
        self.w1, self.w2 = self._initialize_weights()
        self.l1 = l1
        self.l2 = l2
        self.epochs = epochs
        self.eta = eta
        self.alpha = alpha
        self.decrease_const = decrease_const
        self.shuffle = shuffle
        self.minibatches = minibatches
        
        # The dropout variable is the keep probability for neurons.
        # Initialize the class variable "dropout" like other variables above. Also, initialize a variable mask to None.
        # This will allow sharing dropout information during forward and backward pass of the neural networks. Note 
        # that the __init__ function has already been modified to include dropout coefficient as an argument. (0.5 points)
        
        #TODO: Implement

    def _encode_labels(self, y, k):    
        """Encode the labels using one-hot representation

        Parameters
        ------------
        y : y represents target values.

        Returns
        -----------
        onehot array

        """
       
        onehot = np.zeros((k, y.shape[0]))
        for idx, val in enumerate(y):
            onehot[val, idx] = 1.0
        
        return onehot

    def _initialize_weights(self):
        """Initialize weights with small random numbers."""
        
        w1 = self.r.uniform(-1.0, 1.0, size=self.n_hidden*(self.n_features + 1))
        w1 = w1.reshape(self.n_hidden, self.n_features + 1)
        w2 = self.r.uniform(-1.0, 1.0, size=self.n_output*(self.n_hidden + 1))
        w2 = w2.reshape(self.n_output, self.n_hidden + 1)
        
        return w1, w2

    def _sigmoid(self, z):
        """Compute logistic function (sigmoid)

        Uses scipy.special.expit to avoid overflow
        error for very small input values z.

        """
        
        return expit(z)

    def _sigmoid_gradient(self, z):
        """Compute gradient of the logistic function"""
        
        sg = self._sigmoid(z)
        return sg * (1.0 - sg)

    def _add_bias_unit(self, X, how='column'):
        """Add bias unit (column or row of 1s) to array at index 0"""
        
        if how == 'column':
            X_new = np.ones((X.shape[0], X.shape[1] + 1))
            X_new[:, 1:] = X
        elif how == 'row':
            X_new = np.ones((X.shape[0] + 1, X.shape[1]))
            X_new[1:, :] = X
        else:
            raise AttributeError('`how` must be `column` or `row`')
        
        return X_new

    def _feedforward(self, X, w1, w2, if_train=True):
        """Compute feedforward step

        Parameters
        -----------
        X : array, shape = [n_samples, n_features]
            Input layer with original features.
        w1 : array, shape = [n_hidden_units, n_features]
            Weight matrix for input layer -> hidden layer.
        w2 : array, shape = [n_output_units, n_hidden_units]
            Weight matrix for hidden layer -> output layer.
        if_train: if this is during training

        Returns
        ----------
        a1 : array, shape = [n_samples, n_features+1]
            Input values with bias unit.
        z2 : array, shape = [n_hidden, n_samples]
            Net input of hidden layer.
        a2 : array, shape = [n_hidden+1, n_samples]
            Activation of hidden layer.
        z3 : array, shape = [n_output_units, n_samples]
            Net input of output layer.
        a3 : array, shape = [n_output_units, n_samples]
            Activation of output layer.

        """
        
        a1 = self._add_bias_unit(X, how='column')
        z2 = w1.dot(a1.T)
        a2 = self._sigmoid(z2)
        a2 = self._add_bias_unit(a2, how='row')
        
        # Implement inverted dropout using the class variables dropout and activation variable (a2) for the forward
        # pass for the second hidden layer below. To create the mask you will have to use self.r.binomial for 
        # generating the bernoulli distribution. The mask created here needs to be stored in the appropriate mask
        # variable defined in the __init__ function for further use by the backward pass.
        # Make sure to handle the inference case (2.5 points)
        
        # TODO: Implement
        
        z3 = w2.dot(a2)
        a3 = self._sigmoid(z3)
        
        return a1, z2, a2, z3, a3

    def _L2_reg(self, lambda_, w1, w2):
        """Compute L2-regularization cost"""
        
        return (lambda_/2.0) * (np.sum(w1[:, 1:] ** 2) +
                                np.sum(w2[:, 1:] ** 2))

    def _L1_reg(self, lambda_, w1, w2):
        """Compute L1-regularization cost"""
        
        return (lambda_/2.0) * (np.abs(w1[:, 1:]).sum() +
                                np.abs(w2[:, 1:]).sum())

    def _get_cost(self, y_enc, output, w1, w2):
        """Compute cost function.

        Parameters
        ----------
        y_enc : array, shape = (n_labels, n_samples)
            one-hot encoded class labels.
        output : array, shape = [n_output_units, n_samples]
            Activation of the output layer (feedforward)
        w1 : array, shape = [n_hidden_units, n_features]
            Weight matrix for input layer -> hidden layer.
        w2 : array, shape = [n_output_units, n_hidden_units]
            Weight matrix for hidden layer -> output layer.

        Returns
        ---------
        cost : float
            Regularized cost.

        """
        np.seterr(divide='ignore')
        term1 = -y_enc * (np.log(output))
        term2 = (1.0 - y_enc) * np.log(1.0 - output)
        
        term1[np.isneginf(term1)] = 0
        term2[np.isneginf(term2)] = 0
        
        cost = np.sum(term1 - term2)
        L1_term = self._L1_reg(self.l1, w1, w2)
        L2_term = self._L2_reg(self.l2, w1, w2)
        cost = cost + L1_term + L2_term
        
        return cost

    def _get_gradient(self, a1, a2, a3, z2, y_enc, w1, w2):
        """ Compute gradient step using backpropagation.

        Parameters
        ------------
        a1 : array, shape = [n_samples, n_features+1]
            Input values with bias unit.
        a2 : array, shape = [n_hidden+1, n_samples]
            Activation of hidden layer.
        a3 : array, shape = [n_output_units, n_samples]
            Activation of output layer.
        z2 : array, shape = [n_hidden, n_samples]
            Net input of hidden layer.
        y_enc : array, shape = (n_labels, n_samples)
            one-hot encoded class labels.
        w1 : array, shape = [n_hidden_units, n_features]
            Weight matrix for input layer -> hidden layer.
        w2 : array, shape = [n_output_units, n_hidden_units]
            Weight matrix for hidden layer -> output layer.

        Returns
        ---------
        grad1 : array, shape = [n_hidden_units, n_features]
            Gradient of the weight matrix w1.
        grad2 : array, shape = [n_output_units, n_hidden_units]
            Gradient of the weight matrix w2.
        """
        
        # backpropagation
        sigma3 = a3 - y_enc
        z2 = self._add_bias_unit(z2, how='row')
        sigma2 = w2.T.dot(sigma3) * self._sigmoid_gradient(z2)
        
        # Implement dropout for the backward pass, use class variables for mask and dropout for this task (2.5 points)
        
        # TODO: Implement
        
        sigma2 = sigma2[1:, :]
        grad1 = sigma2.dot(a1)
        grad2 = sigma3.dot(a2.T)

        # regularize
        grad1[:, 1:] += self.l2 * w1[:, 1:]
        grad1[:, 1:] += self.l1 * np.sign(w1[:, 1:])
        grad2[:, 1:] += self.l2 * w2[:, 1:]
        grad2[:, 1:] += self.l1 * np.sign(w2[:, 1:])

        return grad1, grad2

    def predict(self, X):
        """Predict class labels

        Parameters
        -----------
        X : array, shape = [n_samples, n_features]
            Input layer with original features.

        Returns:
        ----------
        y_pred : array, shape = [n_samples]
            Predicted class labels.

        """
        if len(X.shape) != 2:
            raise AttributeError('X must be a [n_samples, n_features] array.\n'
                                 'Use X[:,None] for 1-feature classification,'
                                 '\nor X[[i]] for 1-sample classification')

        #TODO: Call the feedforward to get a1, z2, a2, z3, a3 (0.5 point)
        y_pred = np.argmax(z3, axis=0)
        
        return y_pred

    def fit(self, X, y, print_progress=False, validation_freq=0, X_val=None, y_val=None, data_augmentation=False):
        """ Learn weights from training data.

        Parameters
        -----------
        X : array, shape = [n_samples, n_features]
            Input layer with original features.
        y : array, shape = [n_samples]
            Target class labels.
        print_progress : bool (default: False)
            Prints progress as the number of epochs
            to stderr.
        validation_freq=0, X_val=None, y_val=None: If provided, perform validation check for early stopping (Task 2)
        data_augmentation: bool (default: False)
            If true, perform data augmentation (Task 3)

        Returns:
        ----------
        self

        """
        
        self.cost_ = []
        train_err = []
        test_err = []
        X_data, y_data = X.copy(), y.copy()
        y_enc = self._encode_labels(y, self.n_output)

        delta_w1_prev = np.zeros(self.w1.shape)
        delta_w2_prev = np.zeros(self.w2.shape)

        for i in range(self.epochs):

            # adaptive learning rate
            self.eta /= (1 + self.decrease_const*i)

            if print_progress:
                print('\nEpoch: %d/%d' % (i+1, self.epochs))

            if self.shuffle:
                idx = np.random.permutation(y_data.shape[0])
                X_data, y_enc = X_data[idx], y_enc[:, idx]

            mini = np.array_split(range(y_data.shape[0]), self.minibatches)
            for idx in mini:

                # feedforward
                a1, z2, a2, z3, a3 = self._feedforward(X_data[idx],
                                                       self.w1,
                                                       self.w2)
                cost = self._get_cost(y_enc=y_enc[:, idx],
                                      output=a3,
                                      w1=self.w1,
                                      w2=self.w2)
                self.cost_.append(cost)

                # compute gradient via backpropagation
                grad1, grad2 = self._get_gradient(a1=a1, a2=a2,
                                                  a3=a3, z2=z2,
                                                  y_enc=y_enc[:, idx],
                                                  w1=self.w1,
                                                  w2=self.w2)

                delta_w1, delta_w2 = self.eta * grad1, self.eta * grad2
                self.w1 -= (delta_w1 + (self.alpha * delta_w1_prev))
                self.w2 -= (delta_w2 + (self.alpha * delta_w2_prev))
                delta_w1_prev, delta_w2_prev = delta_w1, delta_w2

        return self, train_err, test_err


---
**Points:** $0.0$ of $6$
**Comments:** None

---

b) Next we are goining to plot the variation of training and test accuracies for variying keep probability values ($p$) denoted by np.arange(0.3, 1.0, 0.3). Explain your findings, how do you observe the effect of keep probability values on accuracies. (3 points)



In [ ]:
#TODO: for each dropout value, create a class instance and call the fit function and print the training and test accuracy in percentage for each value of dropout.

#Initialize the class for each dropout value as follows:

""""
nn = MLP(n_output=10, 
             n_features=X_trainval.shape[1], 
             n_hidden=50, 
             l2=0.0, 
             l1=0.0, 
             epochs=500, 
             eta=0.001,
             alpha=0.001,
             decrease_const=0.00001,
             minibatches=50, 
             shuffle=False,
             random_state=1,
             dropout = dropout)
""""

In [ ]:
import matplotlib.pyplot as plt

# Plot the training error and validation error for p values

# TODO: Implement

plt.tight_layout()
plt.show()

---
**Points:** $0.0$ of $3$
**Comments:** None

---

## 2. Early Stopping$~$ (9 points)

For reference see Wikipedia page: https://en.wikipedia.org/wiki/Early_stopping, and also refer to section 7.8 in Deep Learning book.

*Early stopping* is a form of regualrization that avoids overfitting by using the validation set error as an indication on when to stop the training before the validation set error starts increasing again. 

Goal: To study how increasing neurons of a neural network (model complexity) affects the Early Stopping threshold.

a) First, update the feedforward neural network code from Task 1 to calculate training and validation error at every 100 iterations (also known as validation frequency) of the training scheme. For this update, you will modify the fit function. The signature of the train function in task 1 include the following default values: (validation_freq=0, X_val=None, y_val=None), notice that these arguments need to be provided to calculate the validation error (of the validation set) at every 100 iterations (validation frequency).
Overload the fit function in task 1 to perform the early stopping if these arguments are provided (for information about function overloading, refer to: https://stackabuse.com/overloading-functions-and-operators-in-python. 
The fit function should return train_err and test_err for each step in validation frequency (that are initialized as empty). (3 points)

---
**Points:** $0.0$ of $3.0$
**Comments:** None

---

b) Use this code for a different number of neurons (50, 200) and plot the variation of training and validation error for every 100th iteration up to 700 iterations. Label the axes and legends appropriately in the plots. 
Use the same X_trainval, Y_trainval, X_test, Y_test from task 1.
Use Keep probability of dropout to 1 as in default values (no dropout). 

**Note:** to calculate the validation error we use the test set as a proxy validation set. In general, the correct way to do that would be to keep a separate set for validation and test, the reason for that is that any hyperparameter should be selected based on the validation set, and the test set should be kept for testing the model as unseen data only. Please keep in mind that here we are doing a simplification only for the assignment purpose. 

(3 points)

In [ ]:
#TODO: create two classes instances and fit two times with different neurons number

#TODO: Plots for training and test errors for the 2 models 

---
**Points:** $0.0$ of $3.0$
**Comments:** None

---

c) Using these plots, make suggestions for an early stopping criteria for each hidden layer size. (1 point)

Answer:

d) As the number of neurons are increased, you will observe differences in the early stopping criteria for each hidden layer size. Why do you observe such differences? (1 point)

Answer:

e) How early stopping can be similar to L2 regularization? (1 point)

Answer:

---
**Points:** $0.0$ of $3.0$
**Comments:** None

---

## 3. Data Augmentation$~$ (12 points)

While the performance of neural networks can significantly increase with increasing training data size, collecting a large dataset is hard and needs a lot of work in annotation. Therefore, data augmentation is a widley used technique in machine learning to artificially increase your training set size. In case of images, data augmentation methods can be scaling, rotations, cropping, or mirroring. In this exercise, we will get familiar with some of these techniques and apply them on the MNIST fashion dataset. 

For additional reading, you can refer to the following paper (also known as AlexNet) which is one of the papers that revolutionized neural networks applications in image classification and has used data augmentation to increase training data size:

Krizhevsky, Alex, Ilya Sutskever, and Geoffrey E. Hinton. "Imagenet classification with deep convolutional neural networks." Advances in neural information processing systems. 2012 (http://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf).

**Additional note:** These types of augmentation methods usually work best with a type of neural networks called *Convolutional Neural Networks (CNNs)* becuase these networks can learn features that are invariant under transformations (Wikipedia: https://en.wikipedia.org/wiki/Convolutional_neural_network). However, we will use it here with a simple fully connected network to learn the concept. 


Note: for all the next questions, the argument of the function is a 2d Numpy array of 28x28 (one single MNIST example before reshaping)

a) One common way of data augmentation in images is flipping, here, we will flip the images across the vertical axis. In the following, implement a function that takes an input image and perform vertical flipping. Call your function with a simple  input image and display both the original and transformed images. (0.5 point)


In [ ]:
#TODO: Implement vertical flipping. Use Numpy
def flip_img(one_input_image):
    return flipped_image

b) Another common way is image rotation, here, we will flip the images 90 degrees clock wise. In the following, implement a function that takes a one input image and perform rotation. Call your function with a simple one input image and display both the original and transformed images (0.5 point). 

In [ ]:
#TODO: Implement image rotation by 90 degrees clockwise using Numpy
def rotate_img(one_input_image):
    return rotated_img

c) Perform a function that scales one image by a given scale ratio (>1 to work as a zoom-out), make sure that the scaled image has the same size. Call your function with a simple one input image and a scale ratio of 2, and display both the original and transformed images (0.5).


In [ ]:
#TODO: Implement image scaling
#For this, you can use an image processing library such as: https://scikit-image.org/ (install if needed)
def scale_img(one_input_image, scale_ratio):
    return scaled_img

d) Another used method is translation, here, we want to translate our image to the right or the left only, make sure to do zero padding as needed after the translation. Call your function twice, once with a simple one input image and a translation of 5 pixels to the right, and once with the same input image but with a translation of 5 pixels to the left (1 point).  

In [ ]:
#TODO: Implement image translation. Use Numpy
def translate_img(one_input_image, right_or_left_flag, shift_pixels):
    return translated_img

e) Another way of augmentation is introducing random noise to your training set. Implement a function that takes an input image and adds Gaussian noise of mean 0 and std 0.2 to the original image (0.5 points).

In [ ]:
#TODO: Implement adding Gaussian noise. Use Numpy
def distort_img(one_input_image):
    return distorted_img

f) What are other advantages of data augmentation beside increasing data size? Elaborate on in which cases each type of data augmentation mentioned above could be effective (scaling, translation, etc.) (1.5 point)

Answer: 

g) There are two ways of performing data augmentation. One is offline augmentation where we apply these augmentations before the training and increase the training set size by a factor (that is usually done for small datasets), another way is to apply these augmentations for each mini-batch online during the training (used for larger datasets). We will use here online augmentation in order to not extensively increase the required computation time.

1. Perform a function that takes an input of mini-batch images, and for each image in these images, decide if to perform augmentation or not based on a random flip. 

2. If augmentation is performed, decide which type of augmentation to use based on another random variable (options: flip, scale, rotate, translate, Gaussian distortion). 

3. For scaling, generate a random scaling factor (in range from 1 to 2). 

4. For translation, decide randomly based on another variable if translation is done to left or right, and generate a random transition between 5 and 10 pixels.

Note that the number of observation in the batch will be the same.

(3.5 points)

In [ ]:
#TODO: Implement online_augmentation
def online_augmentation(batch_images):
    return augmented_batch
    

h) Now use the function (online_augmentation) that you implemented in 7, to edit the (fit) function in the class in Task 1. 

Note: Use the same X_trainval, Y_trainval, X_test, Y_test from task 1. However, **reshape** the mini-batches to a 2d image shape before applying augmentation and after that reshape it again to the flatted version. The fit function has data_augmentation parameter (default: False), if true, data augmentation should be performed (without early stopping and dropout).

(1 point)

i) Create an instance of the MLP class with 50 neurons. Run the train function for 1500 epochs. Plot the train and test performance over epochs (1.5 point).

In [ ]:
#TODO: create an instrance of the class
#TODO: run the fit function
#TODO: Plot the train and test performance over epochs

j) Compare the performance of this version of the network with the vanilla-network that you have created in Task 2 (the early stopping without dropout). Do you see any performance increase. Explain why or why not. Based on this dataset, what form of data augmentation you think would be less effective than the others? (1.5 point)

---
**Points:** $0.0$ of $12.0$
**Comments:** None

---

## Submission instructions
You should provide a single Jupyter notebook (.ipynb file) as the solution. Put the names and student ids of your team members below. **Make sure to submit only 1 solution to only 1 tutor.**

- Jane Doe, 123456
- Jane Doe, 123456
- Jane Doe, 123456

## Points: 0.0 of 30.0 points